# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [61]:
class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()

    def diff(self):
        pass

    def __add__(self, other):
        return Addition(self, other)

    def __sub__(self, other):
        return Addition(self, Multiplication(cst(-1), other))

    def __mul__(self, other):
        return Multiplication(self, other)

    def __radd__(self, other):
        return self.__add__(cst(other)) if isinstance(other, (int, float)) else NotImplemented

    def __rmul__(self, other):
        return self.__mul__(cst(other)) if isinstance(other, (int, float)) else NotImplemented


class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument


class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument

    def __str__(self):
        return f"{type(self).__name__} {self.argument}"


class Addition(BinaryOperator):
    def __str__(self):
        return f"{self.levy_argument} + {self.pravy_argument}"

    def diff(self):
        return Addition(self.levy_argument.diff(), self.pravy_argument.diff())


class Multiplication(BinaryOperator):
    def __str__(self):
        return f"({self.levy_argument}) * ({self.pravy_argument})"

    def diff(self):
        return Addition(
            Multiplication(self.levy_argument.diff(), self.pravy_argument),
            Multiplication(self.levy_argument, self.pravy_argument.diff())
        )


class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return str(self.value)

    def diff(self):
        return cst(0)


class identity(UnaryOperator):
    def __init__(self):
        pass

    def __str__(self):
        return "x"

    def diff(self):
        return cst(1)


class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent

    def __str__(self):
        return f"({self.argument})^{self.exponent}"

    def diff(self):
        return Multiplication(
            Multiplication(cst(self.exponent), mocnina(self.argument, self.exponent - 1)),
            self.argument.diff()
        )


class sin(UnaryOperator):
    def __str__(self):
        return f"sin({self.argument}"

    def diff(self):
        return Multiplication(cos(self.argument), self.argument.diff())


class cos(UnaryOperator):
    def __str__(self):
        return f"cos({self.argument})"

    def diff(self):
        return Multiplication(cst(-1), Multiplication(sin(self.argument), self.argument.diff()))


class exp(UnaryOperator):
    def __str__(self):
        return f"exp({self.argument})"

    def diff(self):
        return Multiplication(exp(self.argument), self.argument.diff())


class ln(UnaryOperator):
    def __str__(self):
        return f"ln{self.argument}"

    def diff(self):
        return Multiplication(frc(self.argument), self.argument.diff())


class frc(UnaryOperator):
    def __str__(self):
        return f"1/{self.argument}"

    def diff(self):
        return Multiplication(
            cst(-1), 
            Multiplication(frc(mocnina(self.argument, 2)), self.argument.diff())
        )


In [53]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f1)
print(f1.diff())

sin(x + cos((x) * (x)) + 1
(cos(x + cos((x) * (x)))) * (1 + (-1) * ((sin((x) * (x)) * ((1) * (x) + (x) * (1)))) + 0


můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [57]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f2)
print(f2.diff())


(x)^2 + (2) * (x) + 1 + (2) * ((x)^2)
((2) * ((x)^1)) * (1) + (0) * (x) + (2) * (1) + 0 + (0) * ((x)^2) + (2) * (((2) * ((x)^1)) * (1))


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [62]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f3)
print(f3.diff())


(x)^2 + (2) * (x) + exp(x)
((2) * ((x)^1)) * (1) + (0) * (x) + (2) * (1) + (exp(x)) * (1)


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 

In [26]:
class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()

    def diff(self):
        pass

    def __add__(self, other):
        if isinstance(other, (int, float)):  
            other = cst(other)  
        return Addition(self, other).simplify()

    def __sub__(self, other):
        if isinstance(other, (int, float)):  
            other = cst(other)
        return Addition(self, Multiplication(cst(-1), other)).simplify()

    def __mul__(self, other):
        if isinstance(other, (int, float)):  
            other = cst(other)
        return Multiplication(self, other).simplify()

    def __radd__(self, other):
        return self.__add__(other)

    def __rmul__(self, other):
        return self.__mul__(other)


class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument


class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument

    def __str__(self):
        return f"{type(self).__name__} {self.argument}"


class Addition(BinaryOperator):
    def __str__(self):
        return f"{self.levy_argument} + {self.pravy_argument}"

    def diff(self):
        return Addition(self.levy_argument.diff(), self.pravy_argument.diff()).simplify()

    def simplify(self):
        if isinstance(self.levy_argument, cst) and self.levy_argument.value == 0:
            return self.pravy_argument
        if isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 0:
            return self.levy_argument
        return self


class Multiplication(BinaryOperator):
    def __str__(self):
        return f"({self.levy_argument}) * ({self.pravy_argument})"

    def diff(self):
        return Addition(
            Multiplication(self.levy_argument.diff(), self.pravy_argument),
            Multiplication(self.levy_argument, self.pravy_argument.diff())
        ).simplify()

    def simplify(self):
        if isinstance(self.levy_argument, cst) and self.levy_argument.value == 0:
            return cst(0)
        if isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 0:
            return cst(0)
        if isinstance(self.levy_argument, cst) and self.levy_argument.value == 1:
            return self.pravy_argument
        if isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 1:
            return self.levy_argument
        if isinstance(self.levy_argument, cst) and isinstance(self.pravy_argument, cst):
            return cst(self.levy_argument.value * self.pravy_argument.value)
        if self.levy_argument == self.pravy_argument:
            return mocnina(self.levy_argument, 2)
        return self


class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent

    def __str__(self):
        return f"({self.argument})^{self.exponent}"

    def diff(self):
        return Multiplication(
            Multiplication(cst(self.exponent), mocnina(self.argument, self.exponent - 1)),
            self.argument.diff()
        ).simplify()

    def simplify(self):
        if self.exponent == 0:
            return cst(1)
        if self.exponent == 1:
            return self.argument
        return self


class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return str(self.value)

    def diff(self):
        return cst(0)

    def simplify(self):
        return self


class identity(UnaryOperator):
    def __init__(self):
        pass

    def __str__(self):
        return "x"

    def diff(self):
        return cst(1)


class sin(UnaryOperator):
    def __str__(self):
        return f"sin({self.argument})"

    def diff(self):
        return Multiplication(cos(self.argument), self.argument.diff()).simplify()


class cos(UnaryOperator):
    def __str__(self):
        return f"cos({self.argument})"

    def diff(self):
        return Multiplication(cst(-1), Multiplication(sin(self.argument), self.argument.diff())).simplify()


class exp(UnaryOperator):
    def __str__(self):
        return f"exp({self.argument})"

    def diff(self):
        return Multiplication(exp(self.argument), self.argument.diff()).simplify()


class ln(UnaryOperator):
    def __str__(self):
        return f"ln({self.argument})"

    def diff(self):
        return Multiplication(frc(self.argument), self.argument.diff()).simplify()


class frc(UnaryOperator):
    def __str__(self):
        return f"1/({self.argument})"

    def diff(self):
        return Multiplication(
            cst(-1), 
            Multiplication(frc(mocnina(self.argument, 2)), self.argument.diff())
        ).simplify()

In [27]:
x = identity()

print("Test simplify():")
print(f"Addition(x, 0) ➝ {Addition(x, cst(0)).simplify()}")  # x
print(f"Multiplication(x, 1) ➝ {Multiplication(x, cst(1)).simplify()}")  # x
print(f"Multiplication(x, 0) ➝ {Multiplication(x, cst(0)).simplify()}")  # 0
print(f"Multiplication(2, 3) ➝ {Multiplication(cst(2), cst(3)).simplify()}")  # 6
print(f"mocnina(x, 0) ➝ {mocnina(x, 0).simplify()}")  # 1
print(f"mocnina(x, 1) ➝ {mocnina(x, 1).simplify()}")  # x
print(f"Multiplication(x, x) ➝ {Multiplication(x, x).simplify()}")  # x^2
print()

Test simplify():
Addition(x, 0) ➝ x
Multiplication(x, 1) ➝ x
Multiplication(x, 0) ➝ 0
Multiplication(2, 3) ➝ 6
mocnina(x, 0) ➝ 1
mocnina(x, 1) ➝ x
Multiplication(x, x) ➝ (x)^2



In [29]:
x = identity()

print("Test derivací:")
f1 = sin(x + cos(x * x)) + cst(1)
print(f"f1 = {f1}")  
print(f"f1' = {f1.diff()}")  
print()

f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f"f2 = {f2}")  
print(f"f2' = {f2.diff()}")  
print()

f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f"f3 = {f3}")  
print(f"f3' = {f3.diff()}")  
print()

Test derivací:
f1 = sin(x + cos((x)^2)) + 1
f1' = (cos(x + cos((x)^2))) * (1 + (-1) * ((sin((x)^2)) * ((2) * ((x)^1))))

f2 = (x)^2 + (2) * (x) + 1 + (2) * ((x)^2)
f2' = (2) * ((x)^1) + (0) * (x) + (2) * (1) + (0) * ((x)^2) + (2) * ((2) * ((x)^1))

f3 = (x)^2 + (2) * (x) + exp(x)
f3' = (2) * ((x)^1) + (0) * (x) + (2) * (1) + exp(x)

